In [1]:
import re
import logging
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import pickle
from collections import defaultdict
from data.configs.demo import config
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel, BertModel, BertConfig, AutoModelForSequenceClassification
from utils.Manager import Manager

from models.Embeddings.BERT import BERT_Embedding
from models.Encoders.CNN import CNN_Encoder,CNN_User_Encoder
from models.Encoders.RNN import RNN_Encoder,RNN_User_Encoder
from models.Encoders.MHA import MHA_Encoder, MHA_User_Encoder
from models.Modules.DRM import Matching_Reducer, Identical_Reducer
from models.Rankers.BERT import BERT_Onepass_Ranker, BERT_Original_Ranker
from models.Rankers.CNN import CNN_Ranker
from models.Encoders.Pooling import Attention_Pooling, Average_Pooling
from models.UniLM.modeling import TuringNLRv3Model, TuringNLRv3ForSequenceClassification, relative_position_bucket
from models.UniLM.configuration_tnlrv3 import TuringNLRv3Config

from models.BaseModel import BaseModel

from models.Encoders.BERT import BERT_Encoder
from models.Encoders.Pooling import *

from models.ESM import ESM
from models.TESRec import TESRec
from models.XFormer import XFormer
 
from models.Modules.Attention import MultiheadAttention, get_attn_mask, XSoftmax
torch.set_printoptions(threshold=100000)

In [1]:
3721 in [61252, 42975, 43909, 52894, 26312, 45897, 50809, 1146, 22666, 21776, 2207, 49785, 18627, 32915, 33607, 1106, 46731, 23778, 43631, 4731, 31427, 25757, 52537, 21171, 10422, 22316, 21055, 36712, 26835, 56208, 9117, 19526, 17413, 55720, 50011, 36128, 51634, 50011, 43727, 6958, 12660, 30609, 38928, 55127, 44902, 77, 53399, 51668, 7413, 15223]

False

In [10]:
# m = AutoModel.from_pretrained('bert-base-uncased',cache_dir=config.path + 'bert_cache/')
# m2 = AutoModel.from_pretrained('microsoft/deberta-base',cache_dir=config.path + 'bert_cache/')
# m3 = TuringNLRv3ForSequenceClassification.from_pretrained(config.unilm_path, config=TuringNLRv3Config.from_pretrained(config.unilm_config_path))

# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
# t3 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', cache_dir=config.path + "bert_cache/")

In [2]:
# config.reducer = 'entity'
# config.embedding = 'deberta'
# config.bert = 'microsoft/deberta-base'
# config.device = 0
config.bert = 'longformer'
config.seed = None
manager = Manager(config)

loaders = manager.prepare()
X1 = list(loaders[0])
X2 = list(loaders[1])
x1 = X1[0]
x2 = X2[0]

[2021-10-26 14:51:55,964] INFO (utils.Manager) Hyper Parameters are 
{
    "scale": "demo",
    "mode": "train",
    "batch_size": 5,
    "batch_size_news": 100,
    "batch_size_history": 100,
    "k": 5,
    "threshold": -Infinity,
    "abs_length": 40,
    "signal_length": 100,
    "his_size": 50,
    "cdd_size": 5,
    "impr_size": 10,
    "dropout_p": 0.2,
    "lr": 0.0001,
    "bert_lr": 3e-05,
    "embedding": "bert",
    "encoderN": "cnn",
    "encoderU": "rnn",
    "selector": "sfi",
    "reducer": "matching",
    "ranker": "onepass",
    "pooler": "attn",
    "bert_dim": 768,
    "hidden_dim": 384,
    "base_rank": 0,
    "world_size": 0,
    "seed": null,
    "granularity": "token",
    "debias": false,
    "full_attn": true,
    "descend_history": false,
    "shuffle_pos": false,
    "save_pos": false,
    "sep_his": false,
    "diversify": false,
    "no_dedup": false,
    "segment_embed": false,
    "no_rm_punc": false,
    "fast": false,
    "scheduler": "linear",
    "wa

In [5]:
# embedding = BERT_Embedding(manager)

# encoderN = CNN_Encoder(manager)
# encoderN = RNN_Encoder(manager)
# encoderN = MHA_Encoder(manager)

# encoderU = CNN_User_Encoder(manager)
# encoderU = RNN_User_Encoder(manager)
# encoderU = MHA_User_Encoder(manager)
# encoderU = Attention_Pooling(manager)
# encoderU = Average_Pooling(manager)

# reducer = Matching_Reducer(manager)
# reducer = Identical_Reducer(manager)

# ranker = CNN_Ranker(manager)
# ranker = BERT_Onepass_Ranker(manager)
# ranker = BERT_Original_Ranker(manager)

# model = TESRec(manager, embedding, encoderN, encoderU, reducer).to(manager.device)
# model = ESM(manager, embedding, encoderN, encoderU, reducer, ranker).to(manager.device)
model = XFormer(manager).to(manager.device)
# manager.load(model, 589, strict=False)

In [3]:
model(x1)

NameError: name 'model' is not defined